In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, load_img
from PIL import Image
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Unzipping training and testing data

In [ ]:
import zipfile
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip", 'r') as zip_ref:
    zip_ref.extractall('./train')
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip", 'r') as zip_ref:
    zip_ref.extractall('./test')    

In [ ]:
filenames = os.listdir("./train/train")
categories = []
for f_name in filenames:
    category = f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)
    
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [ ]:
path = os.listdir('./test/test1')
count = 0
names = []
for x in path:
    count += 1
    names.append(x)
print('No. of test examples: ', count)

In [ ]:
df

In [ ]:
cwd = './train/train/'
data = []
labels = []
for i in range(25000):
    image = Image.open(cwd + df.filename[i])
    image = image.resize((128,128))
    image = np.array(image)
    data.append(image)
    labels.append(df.category[i])

In [ ]:
data = np.array(data) # Converting the list into an array
labels = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size = 0.6)

### Creating and training the model

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape = (128,128,3)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(2048, activation = 'relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(128, activation = 'relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(2, activation='sigmoid')
])

model.compile(optimizer='adam',
             loss = 'sparse_categorical_crossentropy',
             metrics=['accuracy'])

mod = model.fit(X_train, y_train, epochs = 28, batch_size = 64)
mod

In [ ]:
model.summary()

### Accuracy on test set

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(mod.history['accuracy'], label = 'accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
test_img = []
for i in range(12500):
    image = Image.open('./test/test1/' + names[i])
    image = image.resize((128,128))
    image = np.array(image)
    test_img.append(image)


In [ ]:
y_predicted = model.predict(X_test)
y_predicted_labels = [np.argmax(i) for i in y_predicted]

In [ ]:
index = 2384
lab = y_predicted_labels[index]
animal='dog'
if lab==0:
    animal='cat'
plt.xlabel(animal)
plt.imshow(test_img[index])